# Transfer Learning

* Prep input
* Instantiate a convolution base
* Freeze all layers
* Add new head to model
* Functional model
* Train and save model

In [1]:
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
tensorflow.keras.__version__

'2.2.4-tf'

## Prep Input 

We will be using a netork called `ResNet50` 

This network already has an entire architecture defined for us

It also has the added benefit of being pretrained, meaning it already works, we just need to tune it to our needs

The default input size for this model is 224x224

In [3]:
input_size = (224, 224)

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'data/train',
    batch_size=64,
    class_mode='categorical',
    target_size=input_size)

Found 12894 images belonging to 10 classes.


In [5]:
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_generator = valid_datagen.flow_from_directory(
    'data/val',
    shuffle=False,
    class_mode='categorical',
    target_size=(input_size)
)

Found 500 images belonging to 10 classes.


## Instantiate a convolutional base

We will instantiate ResNet50 with the following parameters:

* `incude_top=False`
* `weights='imagenet'`

We don't include the top because this archecture was originally trained on `imagenet` not `imagenette`

`imagenet` has 1000 classes so we have to remove the prediction layer and change it to the amount of classes we want

This line will need to download ResNet50 the first time you run it

In [6]:
conv_base = ResNet50(include_top=False, weights='imagenet')

/home/chris/.miniconda/envs/cnn/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


## Freeze convolutional base

For now we will freeze all of the layers included in the convolutional base

The model already has layers that detect features like edges and curves

Usually even things like faces, wheels, wings etc. are included in the weights

Because of this we won't waste our time retraining all of these layers for now

In [7]:
# Freeze all of the base layers
for layer in conv_base.layers:
    layer.trainable = False

## Add new head to model

The original `ResNet50` has two layers added to the top

* GlobalAveragePooling2D
* Dense 

In [8]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)

prediction_layer = layers.Dense(10, activation='softmax', name='fc10')(x)

## Functional model

We will use a functional Model to connect the convolutional base to the prediction layer

In [9]:
model = Model(conv_base.input, prediction_layer)

Now we can take a look at our model (ResNet50 is very large uncomment to view it)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
______________________________________________________________________________________________

## Train and save model

We can train as normal now

You should see a big improvement from our custom cnn from the last notebook

a single epoch should beat our last model trained indefinitely

In [11]:
opt = Adam()

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [13]:
history = model.fit_generator(
    generator=train_generator,
    epochs=3,
    validation_data=valid_generator)

Epoch 1/3
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
202/202 [==============================] - 94s 463ms/step - loss: 0.2389 - accuracy: 0.9318 - val_loss: 0.0998 - val_accuracy: 0.9660
Epoch 2/3
202/202 [==============================] - 81s 403ms/step - loss: 0.0704 - accuracy: 0.9801 - val_loss: 0.0862 - val_accuracy: 0.9720
Epoch 3/3
202/202 [==============================] - 82s 404ms/step - loss: 0.0484 - accuracy: 0.9869 - val_loss: 0.0739 - val_accuracy: 0.9740


In [14]:
model.save('models/ResNet50_Transfer_model.h5')
model.save_weights('models/ResNet50_Transfer_weights.h5')

In [15]:
with open('models/ResNet_50_architecture.json', 'w') as f:
    f.write(model.to_json())

### Take a look at the new graph image

This images will be much deeper and you should also pay attention to some of the skip connections

In [16]:
plot_model(model, to_file='plots/models/ResNet50_custom_model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
